In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pprint
import random
os.chdir('/data7/brandon/research/DA_MVR/occ_net/occupancy_networks')

import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import trimesh
import torch
from im2mesh import config, data
from im2mesh.eval import MeshEvaluator
from im2mesh.utils.io import load_pointcloud

In [ ]:
# to benchmark:
# python -m torch.utils.bottleneck train.py configs/domain_adaptation/chair_DA_DANN_test.yaml --benchmark_mode

# dataloader

In [ ]:
cfg = config.load_config('configs/domain_adaptation/chair_DA_DANN.yaml', 'configs/default.yaml')
train_dataset = config.get_dataset('train', cfg)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=20, num_workers=1, shuffle=True,
    collate_fn=data.collate_remove_none,
    worker_init_fn=data.worker_init_fn)

In [ ]:
for batch in train_loader:
    for key in batch:
        print("{} --> {}".format(key, batch[key].shape))
    imgs_source = batch['inputs'].cpu().detach().numpy()
    imgs_target = batch['inputs_target_domain'].cpu().detach().numpy()
    
    break

In [ ]:
plt.imshow(imgs_source[10].transpose(1,2,0))

# Mesh Eval

In [26]:

def mesh_eval(config_yaml, gpu_number, da, generation_dir):

    cfg = config.load_config(config_yaml, 'configs/default.yaml')
    device = torch.device("cuda:{}".format(gpu_number))
    generation_path = os.path.join(cfg['training']['out_dir'], generation_dir)

    # setting up dataloader
    points_field = data.PointsField(
        cfg['data']['points_iou_file'], 
        unpackbits=cfg['data']['points_unpackbits'],
    )
    pointcloud_field = data.PointCloudField(
        cfg['data']['pointcloud_chamfer_file']
    )
    fields = {
        'points_iou': points_field,
        'pointcloud_chamfer': pointcloud_field,
        'idx': data.IndexField(),
    }
    if da:
        dataset_folder = cfg['data']['uda_path']
        categories_to_use = cfg['data']['uda_classes']
    else:
        dataset_folder = cfg['data']['path']
        categories_to_use = cfg['data']['classes']
    dataset = data.Shapes3dDataset(
        dataset_folder, fields,
        cfg['data']['test_split'],
        categories=categories_to_use)
    test_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1, num_workers=0, shuffle=False)

    # evaluation
    evaluator = MeshEvaluator(n_points=100000)
    for it, data_entry in enumerate((test_loader)):

        # Get index etc.
        idx = data_entry['idx'].item()
        try:
            model_dict = dataset.get_model_dict(idx)
        except AttributeError:
            model_dict = {'model': str(idx), 'category': 'n/a'}
        modelname = model_dict['model']
        category_id = model_dict['category']
        print(modelname, category_id)

        # evaluate
        pointcloud_tgt = data_entry['pointcloud_chamfer'].squeeze(0).numpy()
        normals_tgt = data_entry['pointcloud_chamfer.normals'].squeeze(0).numpy()
        points_tgt = data_entry['points_iou'].squeeze(0).numpy()
        occ_tgt = data_entry['points_iou.occ'].squeeze(0).numpy()
        
        mesh_dir = os.path.join(generation_path, 'meshes')
        mesh_dir = os.path.join(mesh_dir, category_id)
        mesh_file = os.path.join(mesh_dir, '%s.off' % modelname)
        if os.path.exists(mesh_file):
            mesh = trimesh.load(mesh_file, process=False)
            eval_dict_mesh = evaluator.eval_mesh(
                mesh, pointcloud_tgt, normals_tgt, points_tgt, occ_tgt)
            #for metric in ["chamfer-L1", "iou", "normals completeness"]:
            #    print("{}: {}".format(metric,eval_dict_mesh[metric]))
        else:
            print('Warning: mesh does not exist: %s' % mesh_file)


In [27]:
dann_no_bg_yaml = "configs/domain_adaptation/chair_DA_DANN.yaml"
mesh_eval(dann_no_bg_yaml, '6', True, "generation_last")

0001 chair
0002 chair
0003 chair
0004 chair
0005 chair
0006 chair
0007 chair
0008 chair
0009 chair
0010 chair
0011 chair
0012 chair
0013 chair
0014 chair
0015 chair
0016 chair
0017 chair
0018 chair
0019 chair
0020 chair
0021 chair
0022 chair
0023 chair
0024 chair
0025 chair
0026 chair
0027 chair
0028 chair
0029 chair
0030 chair
0031 chair
0032 chair
0033 chair
0034 chair
0035 chair
0036 chair
0037 chair
0038 chair
0039 chair
0040 chair
0041 chair
0042 chair
0043 chair
0044 chair
0045 chair
0046 chair
0047 chair
0048 chair
0049 chair
0050 chair
0051 chair
0052 chair
0053 chair
0054 chair
0055 chair
0056 chair
0057 chair
0058 chair
0059 chair
0060 chair
0061 chair
0062 chair
0063 chair
0064 chair
0065 chair
0066 chair
0067 chair
0068 chair
0069 chair
0070 chair
0071 chair
0072 chair
0073 chair
0074 chair
0075 chair
0076 chair
0077 chair
0078 chair


KeyboardInterrupt: 